<a href="https://colab.research.google.com/github/IsmailAKBOGA/NASA-Nearest-Earth-Objects-1910-2024-globalai_ML_project-/blob/main/Nearest_Earth_Objects(1_%5BGlobalA%C4%B0-supervised%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'nasa-nearest-earth-objects-1910-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5410371%2F8984243%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240916%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240916T160641Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1f845c37465cf71b54d0ef8cf36345119069fd2ff259b9c88f00948e5a9296b512c64fe8030ce262fd4c7beec8e1967302b5dcffcc2764467b6c69ea0595ac4a45f02028eace46909de5c09b822cc79e99963d1c804b1a9e020d9403b53a06697fb5d96847b98e84f867ae6e9f80028acc420c71059bc5377a90ad0a8c10ac8dee16c94098295bf7728840fe0199614406437cbec16afae4e7a364be673febccc87a5f3f6d3132b172d8e566b0f3510315d3051c1e414b24bf2a2b2a8709ef2733754463d632f14a43dfdd04d5e48e9559ea423202a3b9054ef3d98550241027dee222a56281c9849ff52bd9d8d547b9af78412fbbfd2c5c086c03192f8a94ed'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Gerekli Kütüphaneleri Yükledik.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from io import StringIO
from sklearn.impute import SimpleImputer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# csv dosyasını okuduk ve ilk 5 satırı yazdırdık.

In [ ]:
df = pd.read_csv("/kaggle/input/nasa-nearest-earth-objects-1910-2024/nearest-earth-objects(1910-2024).csv")
print(df.head())


# Sütundaki verileri tam sayıya dönüştürdük.

In [ ]:
df['is_hazardous'] = df['is_hazardous'].astype(int)

# Kullanılacak sütunları seçtik.

In [ ]:

x = df[['absolute_magnitude', 'relative_velocity', 'miss_distance']]
y = df['is_hazardous']

# Burada veriyi eğitim ve test setlerine ayırdık.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# Eksik verileri medyan ile doldurduk.

In [ ]:
imputer = SimpleImputer(strategy='median')
x_train_imputed = imputer.fit_transform(x_train)
x_test_imputed = imputer.transform(x_test)


# Lojistik Regrasyon modeli oluşturup modeli eğitim verisi ile eğittik.

In [ ]:
model = LogisticRegression()

model.fit(x_train_imputed, y_train)

LogisticRegression()


# Test için tahminlerde bulunuldu.

In [ ]:

y_pred = model.predict(x_test_imputed)


Modelin doğruluk oranı hesaplandı ve rapor oluşturuldu.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)



# Doğruluk oranı ve sınıflandırma raporunu ekrana yazdırdık.

In [ ]:
print(f"Doğruluk: {accuracy}")
print("Sınıflandırma raporu:")
print(report)

# Modelin katsayıları ve kesme terimlerini ekrana yazdırdık.

In [ ]:
print("Katsayılar:", model.coef_)
print("Kesişim:", model.intercept_)
